In [ ]:
import os
import time
import glob
import pandas as pd  
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

In [ ]:
custom_model_id="HalcrowLithology"

# Set the environment variables
os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"] = original_endpoint
os.environ["AZURE_FORM_RECOGNIZER_KEY"] = original_key
model_id = os.getenv("CUSTOM_BUILT_MODEL_ID", custom_model_id)

In [ ]:
# create your `DocumentAnalysisClient` instance and `AzureKeyCredential` variable
document_analysis_client = DocumentAnalysisClient(endpoint="AZURE_FORM_RECOGNIZER_ENDPOINT", credential=AzureKeyCredential("AZURE_FORM_RECOGNIZER_KEY"))

In [ ]:
# Full Folder Content
endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]
model_id = os.getenv("CUSTOM_BUILT_MODEL_ID", custom_model_id)
dpath=r"C:\Users\test\Data\Lithology\Halcraw1969_PT_Lithology\Training"

i=85
columns = ['Well Name', 'Easting', 'Northing', 'Location', 'Yield', 'Drawdown', 'Conductivity', 'GWL', 'Depth', 'Drill Date','Page']
dataset = pd.DataFrame(columns=columns)

# specify the path to the folder containing the PDF files
pdf_folder = os.path.join(dpath, "Dataset")

# get a list of file paths in the folder
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

# iterate through the files and process them one by one
for path_to_sample_documents in pdf_files:
    # add your code here to process the PDF file
#     print("Processing file:", path_to_sample_documents)


    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    # Make sure your document's type is included in the list of document types the custom model can analyze
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            model_id=model_id, document=f
        )
    result = poller.result()
    # Extracting Well Details
    row = []
    data=result.documents[0].to_dict()['fields']
    for col in columns:
        try:
            row.append(data.get(col)['value'])
        except Exception as e:
            print(f"An exception occurred while processing: {d}")
            print(e)
    row = pd.DataFrame([row],columns=columns)
    dataset = pd.concat([dataset, row], ignore_index=True) 
    dataset.to_csv(os.path.join(dpath, "output\WellsDataset.csv"), index=False)
    
    #Extracting Lithology Logs
    data2=result.documents[0].to_dict()['fields']['Lithology']['value']
    rows = []
    for d in data2:
        try:
            from_val = d['value']['From']['content']
            to_val = d['value']['To']['content']
            desc_val = d['value']['Description']['content']
            row = {'From': from_val, 'To': to_val, 'Description': desc_val}
            rows.append(row)
        except Exception as e:
            print(f"An exception occurred while processing: {d}")
            print(e)

    df = pd.DataFrame(rows)
    output=os.path.join(dpath,'output')
    df.to_csv(f'{output}\\Litholog - p{i}.csv', index=False)
    i=i+1
    time.sleep(1) 

In [ ]:
# Single Files
path_to_sample_documents = os.path.abspath(
        os.path.join(
            os.path.abspath(r"C:\Users\test\Data\Lithology"),
            r"Halcraw1969_PT_Lithology\Training\APID.pdf"
        )
    )

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Make sure your document's type is included in the list of document types the custom model can analyze
with open(path_to_sample_documents, "rb") as f:
    poller = document_analysis_client.begin_analyze_document(
        model_id=model_id, document=f
    )
result = poller.result()
# Extracting Well Details
row = []
data=result.documents[0].to_dict()['fields']
for col in columns:
    try:
        row.append(data.get(col)['value'])
    except Exception as e:
        print(f"An exception occurred while processing: {d}")
        print(e)
row = pd.DataFrame([row],columns=columns)
dataset = pd.concat([dataset, row], ignore_index=True) 
dataset.to_csv(os.path.join(dpath, "output\WellsDataset.csv"), index=False)

#Extracting Lithology Logs
data2=result.documents[0].to_dict()['fields']['Lithology']['value']
rows = []
for d in data2:
    try:
        from_val = d['value']['From']['content']
        to_val = d['value']['To']['content']
        desc_val = d['value']['Description']['content']
        row = {'From': from_val, 'To': to_val, 'Description': desc_val}
        rows.append(row)
    except Exception as e:
        print(f"An exception occurred while processing: {d}")
        print(e)

df = pd.DataFrame(rows)
output=os.path.join(dpath,'output')
df.to_csv(f'{output}\\Litholog - p{i}.csv', index=False)
i=i+1
time.sleep(1) 

In [ ]:

for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))


# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print("\nLines found on page {}".format(page.page_number))
    for line in page.lines:
        print("...Line '{}'".format(line.content))
    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content, word.confidence
            )
        )
    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' and has a confidence of {}".format(
                selection_mark.state, selection_mark.confidence
            )
        )

for i, table in enumerate(result.tables):
    print("\nTable {} can be found on page:".format(i + 1))
    for region in table.bounding_regions:
        print("...{}".format(region.page_number))
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index, cell.column_index, cell.content
            )
        )
print("-----------------------------------")

In [ ]:
#chatGPT 
for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))

    # iterate over tables, lines, and selection marks on each page
    for page in result.pages:
        print("\n---Page {}---".format(page.page_number))
        print("\nLines found:")
        for line in page.lines:
            print("    - Line '{}'".format(line.content))
        print("\nWords found:")
        for word in page.words:
            print(
                "    - Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )
        print("\nSelection marks found:")
        for selection_mark in page.selection_marks:
            print(
                "    - Selection mark is '{}' and has a confidence of {}".format(
                    selection_mark.state, selection_mark.confidence
                )
            )

    print("\nTables found:")
    for i, table in enumerate(result.tables):
        print("    - Table #{}".format(i + 1))
        for cell in table.cells:
            print(
                "        - Cell[{}][{}] has content '{}'".format(
                    cell.row_index, cell.column_index, cell.content
                )
            )
